# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f1d1cf55340>
├── 'a' --> tensor([[1.1511, 1.7563, 1.6044],
│                   [0.0219, 2.6813, 0.9464]])
└── 'x' --> <FastTreeValue 0x7f1d1cf557f0>
    └── 'c' --> tensor([[-1.0857, -0.0778, -1.7967,  0.0504],
                        [-0.2485,  0.1971,  0.0729,  0.0676],
                        [-0.1120, -1.2202,  2.8117, -0.2041]])

In [4]:
t.a

tensor([[1.1511, 1.7563, 1.6044],
        [0.0219, 2.6813, 0.9464]])

In [5]:
%timeit t.a

64.4 ns ± 0.0402 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f1d1cf55340>
├── 'a' --> tensor([[ 0.7363,  2.8601,  1.2424],
│                   [ 0.4406, -1.0648, -0.1145]])
└── 'x' --> <FastTreeValue 0x7f1d1cf557f0>
    └── 'c' --> tensor([[-1.0857, -0.0778, -1.7967,  0.0504],
                        [-0.2485,  0.1971,  0.0729,  0.0676],
                        [-0.1120, -1.2202,  2.8117, -0.2041]])

In [7]:
%timeit t.a = new_value

67 ns ± 0.0453 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[1.1511, 1.7563, 1.6044],
               [0.0219, 2.6813, 0.9464]]),
    x: Batch(
           c: tensor([[-1.0857, -0.0778, -1.7967,  0.0504],
                      [-0.2485,  0.1971,  0.0729,  0.0676],
                      [-0.1120, -1.2202,  2.8117, -0.2041]]),
       ),
)

In [10]:
b.a

tensor([[1.1511, 1.7563, 1.6044],
        [0.0219, 2.6813, 0.9464]])

In [11]:
%timeit b.a

57.4 ns ± 0.0442 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.1733,  0.8535, -0.4858],
               [ 0.1538,  0.9937, -0.4064]]),
    x: Batch(
           c: tensor([[-1.0857, -0.0778, -1.7967,  0.0504],
                      [-0.2485,  0.1971,  0.0729,  0.0676],
                      [-0.1120, -1.2202,  2.8117, -0.2041]]),
       ),
)

In [13]:
%timeit b.a = new_value

504 ns ± 0.291 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

837 ns ± 0.592 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11 µs ± 27.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 938 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

145 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f1d1cf55310>
├── 'a' --> tensor([[[1.1511, 1.7563, 1.6044],
│                    [0.0219, 2.6813, 0.9464]],
│           
│                   [[1.1511, 1.7563, 1.6044],
│                    [0.0219, 2.6813, 0.9464]],
│           
│                   [[1.1511, 1.7563, 1.6044],
│                    [0.0219, 2.6813, 0.9464]],
│           
│                   [[1.1511, 1.7563, 1.6044],
│                    [0.0219, 2.6813, 0.9464]],
│           
│                   [[1.1511, 1.7563, 1.6044],
│                    [0.0219, 2.6813, 0.9464]],
│           
│                   [[1.1511, 1.7563, 1.6044],
│                    [0.0219, 2.6813, 0.9464]],
│           
│                   [[1.1511, 1.7563, 1.6044],
│                    [0.0219, 2.6813, 0.9464]],
│           
│                   [[1.1511, 1.7563, 1.6044],
│                    [0.0219, 2.6813, 0.9464]]])
└── 'x' --> <FastTreeValue 0x7f1d1cf55250>
    └── 'c' --> tensor([[[-1.0857, -0.0778, -1.7967,  0.0504],
           

In [21]:
%timeit t_stack(trees)

33 µs ± 17.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f1d1cf5aaf0>
├── 'a' --> tensor([[1.1511, 1.7563, 1.6044],
│                   [0.0219, 2.6813, 0.9464],
│                   [1.1511, 1.7563, 1.6044],
│                   [0.0219, 2.6813, 0.9464],
│                   [1.1511, 1.7563, 1.6044],
│                   [0.0219, 2.6813, 0.9464],
│                   [1.1511, 1.7563, 1.6044],
│                   [0.0219, 2.6813, 0.9464],
│                   [1.1511, 1.7563, 1.6044],
│                   [0.0219, 2.6813, 0.9464],
│                   [1.1511, 1.7563, 1.6044],
│                   [0.0219, 2.6813, 0.9464],
│                   [1.1511, 1.7563, 1.6044],
│                   [0.0219, 2.6813, 0.9464],
│                   [1.1511, 1.7563, 1.6044],
│                   [0.0219, 2.6813, 0.9464]])
└── 'x' --> <FastTreeValue 0x7f1d1cf5aa30>
    └── 'c' --> tensor([[-1.0857, -0.0778, -1.7967,  0.0504],
                        [-0.2485,  0.1971,  0.0729,  0.0676],
                        [-0.1120, -1.2202,  2.8117, -0.2041],
   

In [23]:
%timeit t_cat(trees)

30.7 µs ± 97.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.3 µs ± 60.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[1.1511, 1.7563, 1.6044],
                [0.0219, 2.6813, 0.9464]],
       
               [[1.1511, 1.7563, 1.6044],
                [0.0219, 2.6813, 0.9464]],
       
               [[1.1511, 1.7563, 1.6044],
                [0.0219, 2.6813, 0.9464]],
       
               [[1.1511, 1.7563, 1.6044],
                [0.0219, 2.6813, 0.9464]],
       
               [[1.1511, 1.7563, 1.6044],
                [0.0219, 2.6813, 0.9464]],
       
               [[1.1511, 1.7563, 1.6044],
                [0.0219, 2.6813, 0.9464]],
       
               [[1.1511, 1.7563, 1.6044],
                [0.0219, 2.6813, 0.9464]],
       
               [[1.1511, 1.7563, 1.6044],
                [0.0219, 2.6813, 0.9464]]]),
    x: Batch(
           c: tensor([[[-1.0857, -0.0778, -1.7967,  0.0504],
                       [-0.2485,  0.1971,  0.0729,  0.0676],
                       [-0.1120, -1.2202,  2.8117, -0.2041]],
              
                      [[-1.0857, -0.0778, 

In [26]:
%timeit Batch.stack(batches)

77.7 µs ± 233 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[1.1511, 1.7563, 1.6044],
               [0.0219, 2.6813, 0.9464],
               [1.1511, 1.7563, 1.6044],
               [0.0219, 2.6813, 0.9464],
               [1.1511, 1.7563, 1.6044],
               [0.0219, 2.6813, 0.9464],
               [1.1511, 1.7563, 1.6044],
               [0.0219, 2.6813, 0.9464],
               [1.1511, 1.7563, 1.6044],
               [0.0219, 2.6813, 0.9464],
               [1.1511, 1.7563, 1.6044],
               [0.0219, 2.6813, 0.9464],
               [1.1511, 1.7563, 1.6044],
               [0.0219, 2.6813, 0.9464],
               [1.1511, 1.7563, 1.6044],
               [0.0219, 2.6813, 0.9464]]),
    x: Batch(
           c: tensor([[-1.0857, -0.0778, -1.7967,  0.0504],
                      [-0.2485,  0.1971,  0.0729,  0.0676],
                      [-0.1120, -1.2202,  2.8117, -0.2041],
                      [-1.0857, -0.0778, -1.7967,  0.0504],
                      [-0.2485,  0.1971,  0.0729,  0.0676],
                     

In [28]:
%timeit Batch.cat(batches)

144 µs ± 217 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

327 µs ± 2.81 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
